In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg 
import cv2
import os

In [2]:
# Function Definition
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

def add_cnn_layer(x, index, weight_shape, bias_shape):
    with tf.name_scope('cnnLayer' + index):
        weights = weight_variable(weight_shape)
        bias = bias_variable(bias_shape)
        conv = tf.nn.conv2d(x, weights, strides=[1, 1, 1, 1], padding='SAME') + bias
        activ = tf.nn.relu(tf.layers.batch_normalization(conv))
        cnn_pool = max_pool_3x3(activ)
    return cnn_pool

def add_fc_layer(x, index, units):
    with tf.name_scope('fcLayer' + index):
        fc = tf.layers.dense(inputs=x, units=units)
        activ = tf.nn.relu(tf.layers.batch_normalization(fc))
    return activ

def alpha_delta(x):
    return tf.multiply(tf.multiply(x, 5), tf.exp(tf.multiply(tf.pow(x, 1.6),-14)))

def alpha(x, phs, phh):
    return tf.subtract(tf.add(x, tf.multiply(phs, alpha_delta(x))), tf.multiply(phh, alpha_delta(1-x)))

def alpha2(x, phs, phh):
    return tf.subtract(tf.add(x, tf.multiply(phs, alpha_delta(x))), tf.multiply(phh * 0.2, alpha_delta(1-x)))

def data_term(res, ori):
#     return tf.multiply(tf.tanh(tf.multiply(100., tf.subtract(res, 0.5))), tf.subtract(res, ori))
    return tf.multiply(tf.sign(tf.subtract(res, 0.5)), tf.subtract(res, ori))
#     return -tf.multiply(tf.multiply(res-ori,tf.sign(tf.maximum(0.,tf.multiply(ori-0.5,res-0.5)))), tf.sign(0.5-ori)) 
#     return (-tf.multiply(tf.multiply(res-ori,tf.sign(tf.maximum(0.,tf.multiply(ori-0.5,res-0.5)))), tf.sign(0.5-ori)) 
#             + tf.multiply(10., tf.sign(tf.maximum(0.,-tf.multiply(ori-0.5,res-0.5)))))
#     return (-(res - ori) * tf.sign(tf.maximum(0., 0.5-ori)) 
#             * tf.sign(tf.maximum(0., 0.5-res)) * tf.sign(tf.maximum(0., res-ori))
#            - (ori - res) * tf.sign(tf.maximum(0., ori-0.5))
#            * tf.sign(tf.maximum(0., res-0.5)) * tf.sign(tf.maximum(0., ori-res)) )

def block_contrast(img, oriSize, padRow, padCol, kernelSize, kernel):
    imgpad = tf.reshape(tf.pad(tf.reshape(img, [oriSize, oriSize]), tf.constant([[padRow, padRow,], [padCol, padCol]]), "SYMMETRIC"), 
                        [-1, oriSize + padRow * 2, oriSize + padCol * 2, 1])
    value = kernel
    init = tf.constant_initializer(value)
    return tf.layers.conv2d(imgpad, filters=1, kernel_size=kernelSize, strides=1, padding='VALID', 
                     use_bias=False, kernel_initializer=init,
                     trainable=False)

def block_smooth_term(oriImgBlockContrast, newImgBlockContrast):
    return tf.square(tf.subtract(oriImgBlockContrast, newImgBlockContrast))

In [3]:
# ExCNet Model
blockSize = 4
imgSize = 128
paraSize = imgSize//blockSize

sess = tf.InteractiveSession()
with tf.name_scope('inputLayer'):
    oriImg = tf.placeholder("float", shape=[None, imgSize, imgSize, 1], name='oriImg')

with tf.name_scope('net'):
    cnn_pool1 = add_cnn_layer(oriImg, '1', [3, 3, 1, 64], [64])
    
    cnn_pool2 = add_cnn_layer(cnn_pool1, '2', [3, 3, 64, 128], [128])
#     pool2 = max_pool_3x3(conv2)
    
    cnn_pool3 = add_cnn_layer(cnn_pool2, '3', [3, 3, 128, 256], [256])
#     pool3 = max_pool_3x3(conv3)
    
    cnn_pool4 = add_cnn_layer(cnn_pool3, '4', [3, 3, 256, 512], [512])
#     pool4 = max_pool_3x3(conv4)
    
    cnn_pool5 = add_cnn_layer(cnn_pool4, '5', [3, 3, 512, 512], [512])
#     pool5 = max_pool_3x3(conv5)
    
    pool5_flat = tf.layers.flatten(cnn_pool5)
    with tf.name_scope('fcLayer'):
        fc1 = add_fc_layer(pool5_flat, '1', 512)
        fc2 = add_fc_layer(fc1, '2', 256)
        fc3 = add_fc_layer(fc2, '3', 128)
    
    with tf.name_scope('paraLayer'):
        paraS = tf.layers.dense(inputs=fc3, units=1)
        paraH = tf.layers.dense(inputs=fc3, units=1)
        
with tf.name_scope('ouputLayer'):
    newImg = alpha(oriImg, paraS, paraH)
    
with tf.name_scope('blockLayer'):
    oriImgBlock = tf.nn.avg_pool(oriImg, ksize=[1, blockSize, blockSize, 1], strides=[1, blockSize, blockSize, 1], padding='SAME')
    newImgBlock = tf.nn.avg_pool(newImg, ksize=[1, blockSize, blockSize, 1], strides=[1, blockSize, blockSize, 1], padding='SAME')

with tf.name_scope('dataTerm'):
    Edata = tf.reduce_sum(data_term(newImgBlock, oriImgBlock))

with tf.name_scope('smoothTerm'):
    with tf.name_scope('rightBlock'):
        oriImgBlockContrast_right = block_contrast(oriImgBlock, oriSize=paraSize, padRow=0, padCol=1, kernelSize=[1, 3], kernel=[0, 1, -1])
        newImgBlockContrast_right = block_contrast(newImgBlock, oriSize=paraSize, padRow=0, padCol=1, kernelSize=[1, 3], kernel=[0, 1, -1])
        smoothTerm_right = block_smooth_term(oriImgBlockContrast_right, newImgBlockContrast_right)
    with tf.name_scope('leftBlock'):
        oriImgBlockContrast_left = block_contrast(oriImgBlock, oriSize=paraSize, padRow=0, padCol=1, kernelSize=[1, 3], kernel=[-1, 1, 0])
        newImgBlockContrast_left = block_contrast(newImgBlock, oriSize=paraSize, padRow=0, padCol=1, kernelSize=[1, 3], kernel=[-1, 1, 0])
        smoothTerm_left = block_smooth_term(oriImgBlockContrast_left, newImgBlockContrast_left)
    with tf.name_scope('upBlock'):
        oriImgBlockContrast_up = block_contrast(oriImgBlock, oriSize=paraSize, padRow=1, padCol=0, kernelSize=[3, 1], kernel=[-1, 1, 0])
        newImgBlockContrast_up = block_contrast(newImgBlock, oriSize=paraSize, padRow=1, padCol=0, kernelSize=[3, 1], kernel=[-1, 1, 0])
        smoothTerm_up = block_smooth_term(oriImgBlockContrast_up, newImgBlockContrast_up)
    with tf.name_scope('downBlock'):
        oriImgBlockContrast_down = block_contrast(oriImgBlock, oriSize=paraSize, padRow=1, padCol=0, kernelSize=[3, 1], kernel=[0, 1, -1])
        newImgBlockContrast_down = block_contrast(newImgBlock, oriSize=paraSize, padRow=1, padCol=0, kernelSize=[3, 1], kernel=[0, 1, -1])
        smoothTerm_down = block_smooth_term(oriImgBlockContrast_down, newImgBlockContrast_down)
    Esmooth = tf.reduce_sum(0.5 * tf.add(tf.add(smoothTerm_right, smoothTerm_left), tf.add(smoothTerm_up, smoothTerm_down)))

with tf.name_scope('energyFunction'):
    En = Edata + 12 * Esmooth

In [4]:
def alpha_delta_numpy(x):
    return x * 5 * np.exp((x ** 1.6) * -14)
def alpha_numpy(x, phs, phh):
    return x + phs * alpha_delta_numpy(x) - phh * 0.2 * alpha_delta_numpy(1-x)

In [5]:
train_step = tf.train.AdamOptimizer(0.001).minimize(En)
sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()

In [6]:
# Input Preprocessing
image = cv2.imread('imgs/490.jpg')
height, width, channel = image.shape
B = image[:,:,0]
G = image[:,:,1]
R = image[:,:,2]
Y = 0.299 * R + 0.587 * G + 0.114 * B
I = 0.596 * R - 0.274 * G - 0.322 * B
Q = 0.211 * R - 0.523 * G + 0.312 * B
base_layer = cv2.ximgproc.guidedFilter(Y.astype('uint8'), Y.astype('uint8'), 
                                       int(0.04*min(height, width)), 800)
# ori_base = tf.reshape(base_layer/255, [1, height, width, 1])
# resize_base = tf.image.resize_images(ori_base, [imgSize, imgSize])
# imgV = resize_base.eval()
imgR = cv2.resize(base_layer/255, (imgSize, imgSize))
imgV = np.zeros((1, imgSize, imgSize, 1))
imgV[0,:,:,0] = imgR

In [7]:
# Model Iteration
for i in range(1000):
    train_step.run(feed_dict={oriImg: imgV})  
    if i%200 == 0:
        loss = En.eval(feed_dict={oriImg: imgV})
        phS = paraS.eval(feed_dict={oriImg: imgV})
        phH = paraH.eval(feed_dict={oriImg: imgV})
#         log = sess.run(merged_summary_op, feed_dict={oriImg: imgV})
#         summary_writer.add_summary(log, i)
#         saver.save(sess,'model/model.ckpt')
        print("train_loss %f"%loss)
        print("phS %f"%phS)
        print("phH %f"%phH)

train_loss 1218961.875000
phS -186.117203
phH -50.844368
train_loss -51.349823
phS 1.329334
phH -0.494942
train_loss -47.279877
phS 1.359800
phH -0.488448
train_loss -52.798325
phS 1.320740
phH -0.490173
train_loss -51.962685
phS 1.328388
phH -0.486325


In [36]:
# Image Postprocessing
new_base = alpha_numpy(base_layer/255, np.float64(phS), np.float64(phH))
ratio = new_base/(base_layer/255+0.00001)

In [37]:
newY = ratio * Y
newI = ratio * I
newQ = ratio * Q
# newR = 1. * newY + 0.956 * newI + 0.621 * newQ
# newG = 1. * newY - 0.272 * newI - 0.647 * newQ
# newB = 1. * newY - 1.106 * newI + 1.703 * newQ
newR = (1. * newY + 0.956 * newI + 0.621 * newQ)+0
newG = (1. * newY - 0.272 * newI - 0.647 * newQ)+0
newB = (1. * newY - 1.106 * newI + 1.703 * newQ)+0
newR[newR<0]=0
newG[newG<0]=0
newB[newB<0]=0
newR[newR>255]=255
newG[newG>255]=255
newB[newB>255]=255
result = image.copy()
result[:,:,0] = newB
result[:,:,1] = newG
result[:,:,2] = newR
cv2.imwrite('results/490.jpg',result)

True

In [6]:
#  Batch Test: Read folder images
path = 'BacklitImages/'
savePath = 'my-imgs/'
groupList = ['3-ok/']
imgV = np.zeros((1, imgSize, imgSize, 1))
for groupInd in range(1):
    readPath = path+groupList[groupInd]
    for filename in os.listdir(readPath):
        print("image: %s"%readPath+filename)
        
        image = cv2.imread(readPath+filename)
        height, width, channel = image.shape
        B = image[:,:,0]
        G = image[:,:,1]
        R = image[:,:,2]
        Y = 0.299 * R + 0.587 * G + 0.114 * B
        I = 0.596 * R - 0.274 * G - 0.322 * B
        Q = 0.211 * R - 0.523 * G + 0.312 * B
        base_layer = cv2.ximgproc.guidedFilter(Y.astype('uint8'), Y.astype('uint8'), 
                                               int(0.04*min(height, width)), 800)
        imgR = cv2.resize(base_layer/255, (imgSize, imgSize))
        imgV[0,:,:,0] = imgR
        
        for i in range(1000):
            train_step.run(feed_dict={oriImg: imgV})  
            if i%200 == 0:
                loss = En.eval(feed_dict={oriImg: imgV})
                phS = paraS.eval(feed_dict={oriImg: imgV})
                phH = paraH.eval(feed_dict={oriImg: imgV})
                print("train_loss: %f"%loss)
                print("phS: %f"%phS)
                print("phH: %f"%phH)
                
        new_base = alpha_numpy(base_layer/255, np.float64(phS), np.float64(phH))
        ratio = new_base/(base_layer/255+0.00001)
        
        newY = ratio * Y
        newI = ratio * I
        newQ = ratio * Q
        newR = (1. * newY + 0.956 * newI + 0.621 * newQ)+0
        newG = (1. * newY - 0.272 * newI - 0.647 * newQ)+0
        newB = (1. * newY - 1.106 * newI + 1.703 * newQ)+0
        newR[newR<0]=0
        newG[newG<0]=0
        newB[newB<0]=0
        newR[newR>255]=255
        newG[newG>255]=255
        newB[newB>255]=255
        result = image.copy()
        result[:,:,0] = newB
        result[:,:,1] = newG
        result[:,:,2] = newR
        cv2.imwrite(savePath+groupList[groupInd]+filename,result)

image: all-imgs/3-ok/1.jpg
train_loss: 11306666.000000
phS: 268.772095
phH: 150.211517
train_loss: -45.872120
phS: 0.341333
phH: 0.622553
train_loss: -45.873955
phS: 0.343009
phH: 0.617278
train_loss: -45.307003
phS: 0.343860
phH: 0.647002
train_loss: -45.874203
phS: 0.344020
phH: 0.614488
image: all-imgs/3-ok/10.jpg
train_loss: -10.963268
phS: 0.223996
phH: 0.127401
train_loss: -10.676664
phS: 0.294660
phH: 0.022837
train_loss: -10.675726
phS: 0.295662
phH: 0.024680
train_loss: -10.964870
phS: 0.289392
phH: 0.025420
train_loss: -10.965772
phS: 0.293746
phH: 0.024133
image: all-imgs/3-ok/100.jpg
train_loss: -12.868038
phS: 0.295884
phH: 0.112282
train_loss: -16.866234
phS: 0.358483
phH: 0.337276
train_loss: -16.866226
phS: 0.358461
phH: 0.337328
train_loss: -16.866219
phS: 0.358443
phH: 0.337395
train_loss: -16.866209
phS: 0.358442
phH: 0.337488
image: all-imgs/3-ok/101.jpg
train_loss: -45.055214
phS: 0.627669
phH: 0.349819
train_loss: -44.658115
phS: 0.716006
phH: 0.187084
train_loss:

train_loss: -14.134968
phS: 0.154825
phH: 0.438945
image: all-imgs/3-ok/125.jpg
train_loss: -15.997999
phS: 0.177667
phH: 0.429161
train_loss: -31.727652
phS: 0.554185
phH: 0.002389
train_loss: -31.727650
phS: 0.554185
phH: 0.002387
train_loss: -31.727650
phS: 0.554185
phH: 0.002387
train_loss: -31.727654
phS: 0.554185
phH: 0.002387
image: all-imgs/3-ok/126.jpg
train_loss: -17.148003
phS: 0.544938
phH: 0.015219
train_loss: -24.932644
phS: 0.419295
phH: 0.164644
train_loss: -24.932632
phS: 0.419295
phH: 0.164644
train_loss: -24.932634
phS: 0.419295
phH: 0.164644
train_loss: -24.932632
phS: 0.419295
phH: 0.164644
image: all-imgs/3-ok/127.jpg
train_loss: -19.539478
phS: 0.417457
phH: 0.185368
train_loss: -37.892303
phS: -0.056268
phH: 0.684454
train_loss: -37.892342
phS: -0.055844
phH: 0.683997
train_loss: -37.892345
phS: -0.055841
phH: 0.683994
train_loss: -37.892345
phS: -0.055839
phH: 0.683993
image: all-imgs/3-ok/128.jpg
train_loss: -32.015293
phS: -0.019618
phH: 0.674421
train_loss: 

train_loss: -33.025139
phS: 0.246880
phH: 0.627394
train_loss: -33.025139
phS: 0.246880
phH: 0.627394
image: all-imgs/3-ok/151.jpg
train_loss: 15.333832
phS: 0.242738
phH: 0.621663
train_loss: -5.441797
phS: 0.137704
phH: 0.205000
train_loss: -5.441794
phS: 0.137708
phH: 0.205002
train_loss: -5.441794
phS: 0.137708
phH: 0.205002
train_loss: -5.441794
phS: 0.137708
phH: 0.205002
image: all-imgs/3-ok/152.jpg
train_loss: -18.654209
phS: 0.137397
phH: 0.207275
train_loss: -20.817125
phS: 0.171432
phH: 0.332894
train_loss: -20.817127
phS: 0.171432
phH: 0.332895
train_loss: -20.817127
phS: 0.171432
phH: 0.332895
train_loss: -20.817127
phS: 0.171432
phH: 0.332895
image: all-imgs/3-ok/153.jpg
train_loss: -12.007597
phS: 0.175927
phH: 0.328901
train_loss: -17.883511
phS: 0.447545
phH: 0.046086
train_loss: -17.883511
phS: 0.447536
phH: 0.046091
train_loss: -17.883511
phS: 0.447536
phH: 0.046091
train_loss: -17.883511
phS: 0.447536
phH: 0.046091
image: all-imgs/3-ok/154.jpg
train_loss: -40.532677

train_loss: -41.341030
phS: 0.419831
phH: 0.794412
train_loss: -41.341064
phS: 0.419848
phH: 0.794460
image: all-imgs/3-ok/178.jpg
train_loss: -11.403252
phS: 0.414416
phH: 0.744450
train_loss: -44.862598
phS: 0.575313
phH: 0.070505
train_loss: -44.862595
phS: 0.575310
phH: 0.070509
train_loss: -44.862595
phS: 0.575310
phH: 0.070509
train_loss: -44.862602
phS: 0.575310
phH: 0.070509
image: all-imgs/3-ok/179.jpg
train_loss: -11.931110
phS: 0.570393
phH: 0.077695
train_loss: -41.554100
phS: 0.135792
phH: 0.814892
train_loss: -41.154926
phS: 0.135557
phH: 0.815482
train_loss: -41.154957
phS: 0.135628
phH: 0.815312
train_loss: -41.154972
phS: 0.135707
phH: 0.815119
image: all-imgs/3-ok/18.jpg
train_loss: 31.723469
phS: 0.151968
phH: 0.787593
train_loss: -27.818609
phS: 0.526784
phH: -0.110482
train_loss: -27.693729
phS: 0.526879
phH: -0.110518
train_loss: -27.693729
phS: 0.526903
phH: -0.110539
train_loss: -27.693727
phS: 0.526832
phH: -0.110508
image: all-imgs/3-ok/180.jpg
train_loss: -29

train_loss: -33.791870
phS: 0.282606
phH: 0.328230
train_loss: -33.875324
phS: 0.294920
phH: 0.316911
train_loss: -33.863968
phS: 0.296383
phH: 0.315306
image: all-imgs/3-ok/203.jpg
train_loss: -26.792091
phS: 0.296457
phH: 0.315223
train_loss: -27.724907
phS: 0.196894
phH: 0.388537
train_loss: -27.702303
phS: 0.169063
phH: 0.408235
train_loss: -27.596134
phS: 0.163241
phH: 0.411986
train_loss: -27.596136
phS: 0.163161
phH: 0.411921
image: all-imgs/3-ok/204.jpg
train_loss: -25.306349
phS: 0.164042
phH: 0.411533
train_loss: -53.077972
phS: 0.645872
phH: 0.044536
train_loss: -50.733677
phS: 0.754165
phH: -0.051934
train_loss: -50.450020
phS: 0.782641
phH: -0.076950
train_loss: -50.460442
phS: 0.792140
phH: -0.084711
image: all-imgs/3-ok/205.jpg
train_loss: 4.073711
phS: 0.794730
phH: -0.086671
train_loss: -33.204380
phS: 0.460568
phH: 0.299704
train_loss: -34.689190
phS: 0.377136
phH: 0.389021
train_loss: -34.727726
phS: 0.357418
phH: 0.408697
train_loss: -34.705151
phS: 0.353619
phH: 0.

...

train_loss: -43.649029
phS: 0.691663
phH: -0.328319
train_loss: -43.649029
phS: 0.691784
phH: -0.328588
train_loss: -43.649033
phS: 0.691792
phH: -0.328606
image: all-imgs/3-ok/411.jpg
train_loss: 9.131699
phS: 0.690345
phH: -0.327835
train_loss: -19.530678
phS: 0.433026
phH: 0.092427
train_loss: -20.243397
phS: 0.427422
phH: 0.127036
train_loss: -20.316128
phS: 0.427557
phH: 0.127283
train_loss: -20.316128
phS: 0.427557
phH: 0.127283
image: all-imgs/3-ok/412.jpg
train_loss: -23.661022
phS: 0.427794
phH: 0.129137
train_loss: -102.919495
phS: 0.634702
phH: 0.807139
train_loss: -27.301132
phS: 0.702693
phH: 0.900016
train_loss: -27.935270
phS: 0.727969
phH: 0.900034
train_loss: -27.935368
phS: 0.730436
phH: 0.900035
image: all-imgs/3-ok/413.jpg
train_loss: 204.585953
phS: 0.728295
phH: 0.899147
train_loss: -2.208382
phS: 0.219636
phH: 0.417987
train_loss: -14.014462
phS: 0.180791
phH: 0.261433
train_loss: -17.285530
phS: 0.209422
phH: 0.172573
train_loss: -18.401661
phS: 0.239231
phH: 0.

train_loss: -68.021210
phS: 0.820758
phH: -0.180666
train_loss: -66.315033
phS: 0.855301
phH: -0.230971
train_loss: -66.316589
phS: 0.858412
phH: -0.235585
train_loss: -66.316589
phS: 0.858411
phH: -0.235663
image: all-imgs/3-ok/438.jpg
train_loss: 31.791525
phS: 0.857558
phH: -0.234440
train_loss: -17.960766
phS: 0.430717
phH: 0.111294
train_loss: -19.144434
phS: 0.358904
phH: 0.147355
train_loss: -19.194851
phS: 0.346285
phH: 0.153842
train_loss: -19.195463
phS: 0.343932
phH: 0.155188
image: all-imgs/3-ok/439.jpg
train_loss: -42.974861
phS: 0.344068
phH: 0.155449
train_loss: -55.339382
phS: 0.702625
phH: 0.041026
train_loss: -52.543968
phS: 0.753989
phH: -0.000925
train_loss: -52.249847
phS: 0.758775
phH: -0.005402
train_loss: -52.249939
phS: 0.759707
phH: -0.006281
image: all-imgs/3-ok/44.jpg
train_loss: 8.725723
phS: 0.759189
phH: -0.008033
train_loss: -20.864437
phS: 0.509275
phH: -0.173185
train_loss: -22.101706
phS: 0.487894
phH: -0.151033
train_loss: -22.234745
phS: 0.486720
ph

train_loss: -29.733234
phS: 0.333352
phH: 0.189923
train_loss: -28.945187
phS: 0.562843
phH: -0.132244
train_loss: -27.942993
phS: 0.571323
phH: -0.161978
train_loss: -27.769226
phS: 0.571851
phH: -0.164172
train_loss: -27.769228
phS: 0.571850
phH: -0.164184
image: all-imgs/3-ok/464.jpg
train_loss: 17.895897
phS: 0.570774
phH: -0.163724
train_loss: -12.271542
phS: 0.315866
phH: 0.062291
train_loss: -12.339481
phS: 0.314885
phH: 0.063534
train_loss: -12.339482
phS: 0.314880
phH: 0.063539
train_loss: -12.339478
phS: 0.314883
phH: 0.063537
image: all-imgs/3-ok/465.jpg
train_loss: -9.814769
phS: 0.314938
phH: 0.065459
train_loss: -46.231339
phS: 0.305777
phH: 0.592067
train_loss: -47.203686
phS: 0.229349
phH: 0.692106
train_loss: -46.795673
phS: 0.155505
phH: 0.716413
train_loss: -46.497646
phS: 0.098712
phH: 0.722434
image: all-imgs/3-ok/466.jpg
train_loss: -35.640358
phS: 0.057198
phH: 0.724596
train_loss: -38.764797
phS: 0.288799
phH: 0.707411
train_loss: -40.201340
phS: 0.422015
phH: 0

image: all-imgs/3-ok/53.jpg
train_loss: -11.616117
phS: 0.140844
phH: 0.449722
train_loss: -61.377266
phS: 0.627167
phH: -0.047385
train_loss: -56.395679
phS: 0.713248
phH: -0.260769
train_loss: -50.942879
phS: 0.739084
phH: -0.356086
train_loss: -47.355473
phS: 0.746413
phH: -0.394927
image: all-imgs/3-ok/54.jpg
train_loss: -15.299881
phS: 0.749232
phH: -0.407116
train_loss: -32.054218
phS: 0.700485
phH: -0.136012
train_loss: -37.165695
phS: 0.657326
phH: -0.014006
train_loss: -38.466187
phS: 0.632730
phH: 0.042984
train_loss: -39.438892
phS: 0.623698
phH: 0.068071
image: all-imgs/3-ok/55.jpg
train_loss: -14.772602
phS: 0.622492
phH: 0.075559
train_loss: -30.337887
phS: 0.335637
phH: 0.327414
train_loss: -30.768606
phS: 0.275654
phH: 0.386245
train_loss: -30.701876
phS: 0.263276
phH: 0.397729
train_loss: -30.702232
phS: 0.261568
phH: 0.399319
image: all-imgs/3-ok/56.jpg
train_loss: -11.693253
phS: 0.261288
phH: 0.399164
train_loss: -31.095949
phS: 0.411910
phH: -0.018858
train_loss: -

image: all-imgs/3-ok/8.jpg
train_loss: -15.112968
phS: 0.367113
phH: 0.131287
train_loss: -18.297764
phS: 0.271707
phH: 0.213222
train_loss: -18.473309
phS: 0.261636
phH: 0.244083
train_loss: -18.485977
phS: 0.258383
phH: 0.253082
train_loss: -18.498476
phS: 0.258780
phH: 0.253736
image: all-imgs/3-ok/80.jpg
train_loss: -23.502689
phS: 0.260245
phH: 0.253606
train_loss: -31.739370
phS: 0.567668
phH: 0.017849
train_loss: -30.560114
phS: 0.583903
phH: -0.077901
train_loss: -29.649717
phS: 0.590860
phH: -0.114232
train_loss: -29.583405
phS: 0.593108
phH: -0.125059
image: all-imgs/3-ok/81.jpg
train_loss: -13.338997
phS: 0.593816
phH: -0.128998
train_loss: -23.634151
phS: 0.428526
phH: 0.093250
train_loss: -24.255817
phS: 0.407867
phH: 0.135849
train_loss: -24.317688
phS: 0.404946
phH: 0.141851
train_loss: -24.317732
phS: 0.404465
phH: 0.142603
image: all-imgs/3-ok/82.jpg
train_loss: -18.448503
phS: 0.404087
phH: 0.142587
train_loss: -18.608599
phS: 0.375697
phH: 0.057525
train_loss: -18.49

In [58]:
#  Video Test
imgV = np.zeros((1, imgSize, imgSize, 1))
resultVideo = []

cap = cv2.VideoCapture('video/IMG_6894.MOV')
fps = cap.get(5)
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
#         cv2.imshow('Frame',frame)
        image = frame
        height, width, channel = image.shape
        B = image[:,:,0]
        G = image[:,:,1]
        R = image[:,:,2]
        Y = 0.299 * R + 0.587 * G + 0.114 * B
        I = 0.596 * R - 0.274 * G - 0.322 * B
        Q = 0.211 * R - 0.523 * G + 0.312 * B
        base_layer = cv2.ximgproc.guidedFilter(Y.astype('uint8'), Y.astype('uint8'), 
                                               int(0.04*min(height, width)), 800)
        imgR = cv2.resize(base_layer/255, (imgSize, imgSize))
        imgV[0,:,:,0] = imgR
        
        for i in range(200):
            train_step.run(feed_dict={oriImg: imgV})  
            if i%200 == 0:
                loss = En.eval(feed_dict={oriImg: imgV})
                phS = paraS.eval(feed_dict={oriImg: imgV})
                phH = paraH.eval(feed_dict={oriImg: imgV})
                print("train_loss: %f"%loss)
                print("phS: %f"%phS)
                print("phH: %f"%phH)
                
        new_base = alpha_numpy(base_layer/255, np.float64(phS), np.float64(phH))
        ratio = new_base/(base_layer/255+0.00001)
        
        newY = ratio * Y
        newI = ratio * I
        newQ = ratio * Q
        newR = (1. * newY + 0.956 * newI + 0.621 * newQ)+0
        newG = (1. * newY - 0.272 * newI - 0.647 * newQ)+0
        newB = (1. * newY - 1.106 * newI + 1.703 * newQ)+0
        newR[newR<0]=0
        newG[newG<0]=0
        newB[newB<0]=0
        newR[newR>255]=255
        newG[newG>255]=255
        newB[newB>255]=255
        result = image.copy()
        result[:,:,0] = newB
        result[:,:,1] = newG
        result[:,:,2] = newR
        resultVideo.append(result)
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break       
    else:
        break
        
# cap.release()
cv2.destroyAllWindows()

train_loss: 78.050453
phS: 0.941537
phH: -0.285830
train_loss: -65.613602
phS: 0.872313
phH: -0.658709
train_loss: -72.415375
phS: 0.872147
phH: -0.639246
train_loss: -77.787857
phS: 0.872335
phH: -0.708111
train_loss: -69.500313
phS: 0.873502
phH: -0.740080
train_loss: -71.969666
phS: 0.873904
phH: -0.779955
train_loss: -71.500771
phS: 0.874219
phH: -0.821542
train_loss: -69.518089
phS: 0.874063
phH: -0.868039
train_loss: -76.095955
phS: 0.873073
phH: -0.927891
train_loss: -78.883377
phS: 0.872623
phH: -0.979533
train_loss: -85.468552
phS: 0.872564
phH: -0.975083
train_loss: -82.292007
phS: 0.873157
phH: -0.977345
train_loss: -72.033875
phS: 0.873785
phH: -1.004431
train_loss: -81.308701
phS: 0.873697
phH: -0.924410
train_loss: -72.052856
phS: 0.874685
phH: -0.914046
train_loss: -73.023178
phS: 0.874526
phH: -0.961280
train_loss: -80.593086
phS: 0.874938
phH: -0.932561
train_loss: -70.971375
phS: 0.878259
phH: -0.898527
train_loss: -65.292847
phS: 0.879636
phH: -0.853788
train_loss: -

train_loss: -58.713196
phS: 0.889984
phH: -0.735265
train_loss: -58.599281
phS: 0.891741
phH: -0.808293
train_loss: -51.462311
phS: 0.893859
phH: -0.859101
train_loss: -55.642960
phS: 0.891508
phH: -0.797872
train_loss: -52.507744
phS: 0.892825
phH: -0.803772
train_loss: -47.894066
phS: 0.893088
phH: -0.757201
train_loss: -54.797100
phS: 0.891354
phH: -0.787339
train_loss: -56.192513
phS: 0.892556
phH: -0.778214
train_loss: -48.891174
phS: 0.894272
phH: -0.721252
train_loss: -58.321198
phS: 0.892220
phH: -0.740229
train_loss: -50.249569
phS: 0.895323
phH: -0.701561
train_loss: -57.204781
phS: 0.893768
phH: -0.745007
train_loss: -56.061790
phS: 0.894311
phH: -0.784663
train_loss: -51.331749
phS: 0.896688
phH: -0.770854
train_loss: -58.566948
phS: 0.893585
phH: -0.827929
train_loss: -54.881516
phS: 0.897194
phH: -0.836753
train_loss: -43.030205
phS: 0.899793
phH: -0.741325
train_loss: -61.599228
phS: 0.896412
phH: -0.760579
train_loss: -53.646584
phS: 0.898664
phH: -0.769273
train_loss: 

train_loss: -49.650772
phS: 1.021249
phH: -0.688277
train_loss: -55.799068
phS: 1.019464
phH: -0.662159
train_loss: -46.004044
phS: 1.029895
phH: -0.784183
train_loss: -46.115021
phS: 1.021883
phH: -0.786076
train_loss: -52.389305
phS: 1.010566
phH: -0.808936
train_loss: -51.697433
phS: 1.009823
phH: -0.719651
train_loss: -51.773460
phS: 1.013257
phH: -0.850828
train_loss: -53.220993
phS: 1.015798
phH: -0.807007
train_loss: -49.728615
phS: 1.023353
phH: -0.780010
train_loss: -43.511322
phS: 1.029011
phH: -0.684828
train_loss: -46.828808
phS: 1.026001
phH: -0.639738
train_loss: -49.899559
phS: 1.026304
phH: -0.595463
train_loss: -44.590446
phS: 1.037436
phH: -0.652139
train_loss: -41.949394
phS: 1.037139
phH: -0.686923
train_loss: -44.502129
phS: 1.030647
phH: -0.697768
train_loss: -52.855583
phS: 1.023070
phH: -0.672531
train_loss: -46.237785
phS: 1.029053
phH: -0.718847
train_loss: -50.551781
phS: 1.025524
phH: -0.821628
train_loss: -43.998619
phS: 1.031462
phH: -0.884756
train_loss: 

In [59]:
for frame in resultVideo:
    cv2.imshow('result',frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break 
cv2.destroyAllWindows()

In [60]:
# cap = cv2.VideoCapture('video/IMG_6847.MOV')
fps = cap.get(5)
width = int(cap.get(3))
height = int(cap.get(4))
out = cv2.VideoWriter('video/NEW_IMG_6894.mp4',cv2.VideoWriter_fourcc('M','P','4','V'), fps, (width,height))
for frame in resultVideo:
    out.write(frame)

cap.release()
out.release()

In [70]:
cap = cv2.VideoCapture('video/NEW_IMG_6894.mp4')
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        image = frame
    else:
        break
cap.release()
cv2.imwrite('video/NEW_IMG_6894.png',image)

True